In [187]:
import sys, os
sys.path.append("/opt/livingdata/lib")
from livdattable import *
import openpyxl
import wave
import contextlib



In [188]:
def getaudiolen(fname):
	with contextlib.closing(wave.open(fname,'r')) as f:
		frames = f.getnframes()
		rate = f.getframerate()
		duration = frames / float(rate)
		return duration


def lookupnamefornumber(number,people):
	for person in people.matrix:
			if person['Mob1'].strip(".0")==number or person['Mob2'].strip(".0")==number:
				return person['Name']
            
def getcalllist(callspath):
    calllist=[]
    calls=os.listdir(callspath)
    for call in calls:
        if "calllog" in call:
            break
        i=0
        dictionary={}
        callvals=call.split("-")
        for key in callkeys:
            dictionary[callkeys[i]]=callvals[i]
            i+=1
        dictionary['name']='Unknown'
        dictionary['number']=dictionary['number'].replace("+91","")
        dictionary['name']=lookupnamefornumber(dictionary['number'],people)
        dictionary['date']=dictionary['year']+"-"+dictionary['month']+"-"+dictionary['day']
        dictionary['time']=dictionary['hour']+":"+dictionary['min']+":"+dictionary['sec']
        callfiles=os.listdir(os.path.join(callspath,call))
        dictionary['recording']="None"
        dictionary['recordlen']="None"
        for callfile in callfiles:
            if "callfile" in callfile:
                dictionary['recording']=callfile
                dictionary['recordlen']=getaudiolen(os.path.join(callspath,call,callfile))
        
        calllog=os.path.join(callspath,call,"log",call+".log")
        dictionary['totalduration']=gettotalduration(calllog)
        
        
        dictionary['numinvalids']=getnuminvalids(calllog)
        dictionary['numplayed']=getnumplayed(calllog)
        dictionary['netduration']=getnetduration(dictionary['totalduration'])
        if dictionary['recordlen'] != "None":
            dictionary['totalduration']+=float(dictionary['recordlen'])
            dictionary['netduration']+=float(dictionary['recordlen'])-20
        calllist.append(dictionary)
    return calllist

def gettotalduration(calllog):
    
    f=open(calllog,"r")
    logtext=f.read().replace("Starting Logfile","").strip("\n")
    lines=logtext.split("\n")
    length=len(lines)
    try:
        starttime=datetime.datetime.strptime(lines[0].split(" - ")[0].split(",")[0],"%Y-%m-%d %H:%M:%S")
        stoptime=datetime.datetime.strptime(lines[length-2].split(" - ")[0].split(",")[0],"%Y-%m-%d %H:%M:%S")
        print calllog,starttime,stoptime
        return float((stoptime-starttime).total_seconds())
    except:
        print calllog,lines
        return 0
    
def getnuminvalids(calllog):
    op=os.popen("cat %s | grep invalid | wc -l" %calllog).read().strip("\n")
    return int(op)
def getnumplayed(calllog):
    return ""
def getnetduration(duration):
    print duration
    if duration < 24:
        return duration
    if duration > 24 and duration < 40:
        return duration - 24
    else:
        return duration - 40

In [189]:
callspath="/home/arjun/calls"
outfilename="/home/arjun/SKSFullReportDec2017.xlsx"
e=ExcelFile()
e.importascsv("/home/arjun/MasterSKSList.xlsx")
people=e.worksheets[0]
promptsdir="/home/arjun/dev/et-mojobol-workflows/voh/vohprompts/"
prompts=os.listdir(promptsdir)


In [190]:
callkeys=["number","server","year","month","day","hour","min","sec"]
calllist=getcalllist(callspath)

t=Table()
t.colnames=callkeys
t.colnames+=["name","date","time","recording","recordlen","totalduration","numinvalids","numplayed","netduration"]

t.matrix=calllist

/home/arjun/calls/+919410112284-ShardaKeSwar-2017-Apr-09-18-05-07/log/+919410112284-ShardaKeSwar-2017-Apr-09-18-05-07.log 2017-04-09 18:05:07 2017-04-09 18:05:07
0.0
/home/arjun/calls/+917500340046-ShardaKeSwar-2017-Nov-28-14-58-03/log/+917500340046-ShardaKeSwar-2017-Nov-28-14-58-03.log 2017-11-28 14:58:03 2017-11-28 14:58:48
45.0
/home/arjun/calls/+918192991809-ShardaKeSwar-2017-Aug-05-13-13-53/log/+918192991809-ShardaKeSwar-2017-Aug-05-13-13-53.log 2017-08-05 13:13:53 2017-08-05 13:14:55
62.0
/home/arjun/calls/7830773329-ShardaKeSwar-2017-Dec-04-13-16-23/log/7830773329-ShardaKeSwar-2017-Dec-04-13-16-23.log 2017-12-04 13:16:23 2017-12-04 13:17:57
94.0
/home/arjun/calls/+918989161881-ShardaKeSwar-2017-Nov-20-11-28-28/log/+918989161881-ShardaKeSwar-2017-Nov-20-11-28-28.log 2017-11-20 11:28:28 2017-11-20 11:28:28
0.0
/home/arjun/calls/+918171339378-ShardaKeSwar-2017-Mar-17-15-41-55/log/+918171339378-ShardaKeSwar-2017-Mar-17-15-41-55.log 2017-03-17 15:41:55 2017-03-17 15:42:52
57.0
/home/

0.0
/home/arjun/calls/+911408799774-ShardaKeSwar-2017-Apr-29-12-47-33/log/+911408799774-ShardaKeSwar-2017-Apr-29-12-47-33.log 2017-04-29 12:47:33 2017-04-29 12:47:40
7.0
/home/arjun/calls/8989161881-ShardaKeSwar-2017-Apr-04-14-44-09/log/8989161881-ShardaKeSwar-2017-Apr-04-14-44-09.log 2017-04-04 14:44:09 2017-04-04 14:44:09
0.0
/home/arjun/calls/+917088543117-ShardaKeSwar-2017-Sep-14-11-43-50/log/+917088543117-ShardaKeSwar-2017-Sep-14-11-43-50.log 2017-09-14 11:43:50 2017-09-14 11:44:54
64.0
/home/arjun/calls/Unknown-MojoBolSampleServer-2017-Feb-24-14-47-48/log/Unknown-MojoBolSampleServer-2017-Feb-24-14-47-48.log 2017-02-24 14:47:48 2017-02-24 14:48:04
16.0
/home/arjun/calls/+919457935097-ShardaKeSwar-2017-Sep-26-10-01-38/log/+919457935097-ShardaKeSwar-2017-Sep-26-10-01-38.log 2017-09-26 10:01:38 2017-09-26 10:01:38
0.0
/home/arjun/calls/+919411129984-ShardaKeSwar-2017-Jun-29-22-23-14/log/+919411129984-ShardaKeSwar-2017-Jun-29-22-23-14.log 2017-06-29 22:23:14 2017-06-29 22:24:00
46.0
/

/home/arjun/calls/8989161881-ShardaKeSwar-2017-Apr-04-14-37-17/log/8989161881-ShardaKeSwar-2017-Apr-04-14-37-17.log 2017-04-04 14:37:17 2017-04-04 14:37:17
0.0
/home/arjun/calls/+918076014579-ShardaKeSwar-2017-Aug-23-09-56-18/log/+918076014579-ShardaKeSwar-2017-Aug-23-09-56-18.log 2017-08-23 09:56:18 2017-08-23 09:56:18
0.0
/home/arjun/calls/+919958430737-ShardaKeSwar-2017-Mar-21-13-00-46/log/+919958430737-ShardaKeSwar-2017-Mar-21-13-00-46.log 2017-03-21 13:00:46 2017-03-21 13:02:03
77.0
/home/arjun/calls/+919756470767-ShardaKeSwar-2017-May-30-15-23-17/log/+919756470767-ShardaKeSwar-2017-May-30-15-23-17.log 2017-05-30 15:23:17 2017-05-30 15:24:45
88.0
/home/arjun/calls/+911129533957-ShardaKeSwar-2017-Jul-14-13-01-17/log/+911129533957-ShardaKeSwar-2017-Jul-14-13-01-17.log 2017-07-14 13:01:17 2017-07-14 13:02:51
94.0
/home/arjun/calls/+918076014579-ShardaKeSwar-2017-Dec-13-14-02-43/log/+918076014579-ShardaKeSwar-2017-Dec-13-14-02-43.log 2017-12-13 14:02:43 2017-12-13 14:04:17
94.0
/home/

/home/arjun/calls/+919410163343-ShardaKeSwar-2017-Jul-22-20-03-27/log/+919410163343-ShardaKeSwar-2017-Jul-22-20-03-27.log 2017-07-22 20:03:27 2017-07-22 20:08:09
282.0
/home/arjun/calls/+919557077247-MojoBolSampleServer-2017-Mar-02-14-21-11/log/+919557077247-MojoBolSampleServer-2017-Mar-02-14-21-11.log 2017-03-02 14:21:11 2017-03-02 14:21:44
33.0
/home/arjun/calls/+918076014579-ShardaKeSwar-2017-Nov-27-11-08-29/log/+918076014579-ShardaKeSwar-2017-Nov-27-11-08-29.log 2017-11-27 11:08:29 2017-11-27 11:10:03
94.0
/home/arjun/calls/+919424989823-MojoBolSampleServer-2017-Feb-28-13-30-10/log/+919424989823-MojoBolSampleServer-2017-Feb-28-13-30-10.log 2017-02-28 13:30:10 2017-02-28 13:30:55
45.0
/home/arjun/calls/+919458356124-ShardaKeSwar-2017-Sep-05-13-07-55/log/+919458356124-ShardaKeSwar-2017-Sep-05-13-07-55.log 2017-09-05 13:07:55 2017-09-05 13:08:47
52.0
/home/arjun/calls/+911129533957-ShardaKeSwar-2017-Sep-19-16-34-18/log/+911129533957-ShardaKeSwar-2017-Sep-19-16-34-18.log 2017-09-19 16:

94.0
/home/arjun/calls/+919424989823-ShardaKeSwar-2017-Aug-31-07-05-23/log/+919424989823-ShardaKeSwar-2017-Aug-31-07-05-23.log 2017-08-31 07:05:23 2017-08-31 07:06:50
87.0
/home/arjun/calls/+918076014579-ShardaKeSwar-2017-Nov-10-16-45-32/log/+918076014579-ShardaKeSwar-2017-Nov-10-16-45-32.log 2017-11-10 16:45:32 2017-11-10 16:46:33
61.0
/home/arjun/calls/+917302206127-ShardaKeSwar-2017-Sep-14-08-43-41/log/+917302206127-ShardaKeSwar-2017-Sep-14-08-43-41.log 2017-09-14 08:43:41 2017-09-14 08:46:40
179.0
/home/arjun/calls/7830169122-ShardaKeSwar-2017-May-03-11-30-47/log/7830169122-ShardaKeSwar-2017-May-03-11-30-47.log 2017-05-03 11:30:47 2017-05-03 11:31:12
25.0
/home/arjun/calls/+918076014579-ShardaKeSwar-2017-Aug-12-23-26-01/log/+918076014579-ShardaKeSwar-2017-Aug-12-23-26-01.log 2017-08-12 23:26:01 2017-08-12 23:28:37
156.0
/home/arjun/calls/+917599084387-ShardaKeSwar-2017-Sep-05-18-18-12/log/+917599084387-ShardaKeSwar-2017-Sep-05-18-18-12.log 2017-09-05 18:18:12 2017-09-05 18:18:12
0.

94.0
/home/arjun/calls/+918958381627-ShardaKeSwar-2017-Apr-14-08-19-11/log/+918958381627-ShardaKeSwar-2017-Apr-14-08-19-11.log 2017-04-14 08:19:11 2017-04-14 08:19:32
21.0
/home/arjun/calls/+919752226251-ShardaKeSwar-2017-Apr-19-17-42-31/log/+919752226251-ShardaKeSwar-2017-Apr-19-17-42-31.log 2017-04-19 17:42:31 2017-04-19 17:47:03
272.0
/home/arjun/calls/+919012022211-ShardaKeSwar-2017-Aug-12-04-23-03/log/+919012022211-ShardaKeSwar-2017-Aug-12-04-23-03.log 2017-08-12 04:23:03 2017-08-12 04:23:03
0.0
/home/arjun/calls/+919012022211-ShardaKeSwar-2017-May-23-15-04-27/log/+919012022211-ShardaKeSwar-2017-May-23-15-04-27.log 2017-05-23 15:04:27 2017-05-23 15:04:27
0.0
/home/arjun/calls/+919711878120-ShardaKeSwar-2017-Jul-31-20-39-20/log/+919711878120-ShardaKeSwar-2017-Jul-31-20-39-20.log 2017-07-31 20:39:20 2017-07-31 20:40:14
54.0
/home/arjun/calls/samplesipuser-MojoBolSampleServer-2017-Mar-02-14-28-36/log/samplesipuser-MojoBolSampleServer-2017-Mar-02-14-28-36.log 2017-03-02 14:28:36 2017-

53.0
/home/arjun/calls/+911129533957-ShardaKeSwar-2017-Jun-05-11-37-09/log/+911129533957-ShardaKeSwar-2017-Jun-05-11-37-09.log 2017-06-05 11:37:09 2017-06-05 11:38:07
58.0
/home/arjun/calls/+919411129984-ShardaKeSwar-2017-Jun-28-12-57-33/log/+919411129984-ShardaKeSwar-2017-Jun-28-12-57-33.log 2017-06-28 12:57:33 2017-06-28 12:57:33
0.0
/home/arjun/calls/+919958430737-ShardaKeSwar-2017-Jul-15-10-31-21/log/+919958430737-ShardaKeSwar-2017-Jul-15-10-31-21.log 2017-07-15 10:31:21 2017-07-15 10:32:55
94.0
/home/arjun/calls/9012014890-ShardaKeSwar-2017-Apr-03-18-28-26/log/9012014890-ShardaKeSwar-2017-Apr-03-18-28-26.log 2017-04-03 18:28:26 2017-04-03 18:28:34
8.0
/home/arjun/calls/+919910810032-MojoBolSampleServer-2017-Mar-03-13-44-07/log/+919910810032-MojoBolSampleServer-2017-Mar-03-13-44-07.log 2017-03-03 13:44:07 2017-03-03 13:44:33
26.0
/home/arjun/calls/+917599084387-ShardaKeSwar-2017-Sep-06-10-03-54/log/+917599084387-ShardaKeSwar-2017-Sep-06-10-03-54.log 2017-09-06 10:03:54 2017-09-06 1

/home/arjun/calls/+918989161881-ShardaKeSwar-2017-Nov-10-16-36-04/log/+918989161881-ShardaKeSwar-2017-Nov-10-16-36-04.log 2017-11-10 16:36:04 2017-11-10 16:36:38
34.0
/home/arjun/calls/samplesipuser-MojoBolSampleServer-2017-Feb-23-00-42-25/log/samplesipuser-MojoBolSampleServer-2017-Feb-23-00-42-25.log 2017-02-23 00:42:25 2017-02-23 00:42:28
3.0
/home/arjun/calls/+919639349064-ShardaKeSwar-2017-Sep-17-13-58-56/log/+919639349064-ShardaKeSwar-2017-Sep-17-13-58-56.log 2017-09-17 13:58:56 2017-09-17 14:05:56
420.0
/home/arjun/calls/9917324658-ShardaKeSwar-2017-Mar-20-15-42-17/log/9917324658-ShardaKeSwar-2017-Mar-20-15-42-17.log 2017-03-20 15:42:17 2017-03-20 15:42:17
0.0
/home/arjun/calls/+918006397040-ShardaKeSwar-2017-Nov-21-17-36-57/log/+918006397040-ShardaKeSwar-2017-Nov-21-17-36-57.log 2017-11-21 17:36:57 2017-11-21 17:38:31
94.0
/home/arjun/calls/+919411758351-ShardaKeSwar-2017-May-03-14-33-13/log/+919411758351-ShardaKeSwar-2017-May-03-14-33-13.log 2017-05-03 14:33:13 2017-05-03 14:33

0.0
/home/arjun/calls/+917249929294-ShardaKeSwar-2017-Dec-10-11-27-37/log/+917249929294-ShardaKeSwar-2017-Dec-10-11-27-37.log 2017-12-10 11:27:37 2017-12-10 11:27:37
0.0
/home/arjun/calls/8057996935-ShardaKeSwar-2017-May-03-13-52-49/log/8057996935-ShardaKeSwar-2017-May-03-13-52-49.log 2017-05-03 13:52:49 2017-05-03 13:53:14
25.0
/home/arjun/calls/+919910810032-MojoBolSampleServer-2017-Mar-02-08-23-08/log/+919910810032-MojoBolSampleServer-2017-Mar-02-08-23-08.log 2017-03-02 08:23:08 2017-03-02 08:23:41
33.0
/home/arjun/calls/+918954945002-ShardaKeSwar-2017-May-03-13-53-32/log/+918954945002-ShardaKeSwar-2017-May-03-13-53-32.log 2017-05-03 13:53:32 2017-05-03 13:54:41
69.0
/home/arjun/calls/8954945002-ShardaKeSwar-2017-Dec-04-15-03-41/log/8954945002-ShardaKeSwar-2017-Dec-04-15-03-41.log 2017-12-04 15:03:41 2017-12-04 15:05:15
94.0
/home/arjun/calls/+917599084387-ShardaKeSwar-2017-Sep-05-13-11-44/log/+917599084387-ShardaKeSwar-2017-Sep-05-13-11-44.log 2017-09-05 13:11:44 2017-09-05 13:11:4

In [191]:
calllist

[{'date': '2017-Apr-09',
  'day': '09',
  'hour': '18',
  'min': '05',
  'month': 'Apr',
  'name': None,
  'netduration': 0.0,
  'number': '9410112284',
  'numinvalids': 0,
  'numplayed': '',
  'recording': 'None',
  'recordlen': 'None',
  'sec': '07',
  'server': 'ShardaKeSwar',
  'time': '18:05:07',
  'totalduration': 0.0,
  'year': '2017'},
 {'date': '2017-Nov-28',
  'day': '28',
  'hour': '14',
  'min': '58',
  'month': 'Nov',
  'name': None,
  'netduration': 5.0,
  'number': '7500340046',
  'numinvalids': 0,
  'numplayed': '',
  'recording': 'None',
  'recordlen': 'None',
  'sec': '03',
  'server': 'ShardaKeSwar',
  'time': '14:58:03',
  'totalduration': 45.0,
  'year': '2017'},
 {'date': '2017-Aug-05',
  'day': '05',
  'hour': '13',
  'min': '13',
  'month': 'Aug',
  'name': None,
  'netduration': 85.0,
  'number': '8192991809',
  'numinvalids': 0,
  'numplayed': '',
  'recording': 'callfile-+918192991809-ShardaKeSwar-2017-Aug-05-13-13-53-fcb1d5fc-381f-4495-a623-07b81e3cb149.wav'

In [192]:
promptslen=0
for prompt in prompts:
    length=getaudiolen(os.path.join(promptsdir,prompt))
    if "old" not in prompt:
        promptslen+=length
        print prompt,length,promptslen

welcome.wav 24.546375 24.546375
invalid.wav 2.60925 27.155625
options12.wav 14.286125 41.44175
explanation.wav 2.59125 44.033
thankyourecorded.wav 2.583125 46.616125
recordmessagebeep.wav 17.254 63.870125


In [193]:
x=ExcelFile()
x.worksheetnames.append('All Calls')
x.worksheets.append(t)

In [194]:
callsbynum=Table()
callsbynum.filename="Calls by Number"
callsbynum.colnames=["Number","Number of Calls","Name of Caller","Number of Recordings"]
numlist=[]
for call in t.matrix:
    numlist.append(call['number'].replace("+91",""))
numlist=list(set(numlist))
callmatrix=[]
for num in numlist:
    dictionary={}
    numcalls=0
    numrecs=0
    for call in t.matrix:
        if call['number']==num:
            numcalls+=1
            if call['recording']!="None":
                numrecs+=1
    dictionary['Number']=num
    dictionary['Name of Caller']=lookupnamefornumber(dictionary['Number'],people)
    dictionary['Number of Calls']=numcalls
    dictionary['Number of Recordings']=numrecs
    callmatrix.append(dictionary)
callsbynum.matrix=callmatrix

x.worksheetnames.append("Calls by Number")
x.worksheets.append(callsbynum)
x.exportfile(outfilename)

xfile=openpyxl.load_workbook(outfilename)
callsbyday=xfile.create_sheet("Calls by Day")
callsbyday["A1"]="Day"
callsbyday["B1"]="Number of Calls"
callsbyday["C1"]="Number of Recordings"

days=[]
for call in t.matrix:
    days.append(call['date'])
days=list(set(days))
days.sort()
daycountdict=[]
for day in days:
    daycount=0
    dayrecount=0
    for call in t.matrix:
        if call['date']==day:
            daycount+=1
            if call['recording']!="None":
                dayrecount+=1
    daydict={}
    daydict['Day']=datetime.datetime.strptime(day,"%Y-%b-%d")
    daydict['Number of Calls']=daycount
    daydict["Number of Recordings"]=dayrecount

    daycountdict.append(daydict)
r=2
c=1
for daydict in daycountdict:
    cell=callsbyday.cell(row=r,column=c)
    cell.value=daydict['Day']
    c+=1
    cell=callsbyday.cell(row=r,column=c)
    cell.value=daydict['Number of Calls']
    c+=1
    cell=callsbyday.cell(row=r,column=c)
    cell.value=daydict['Number of Recordings']

    r+=1
    c=1

xfile.save(outfilename)
	

In [164]:
getaudiolen("/home/arjun/calls/+918192991809-ShardaKeSwar-2017-Aug-05-13-13-53/callfile-+918192991809-ShardaKeSwar-2017-Aug-05-13-13-53-fcb1d5fc-381f-4495-a623-07b81e3cb149.wav")

83.0